In [1]:
import pandas as pd
import networkx as nx

df = pd.read_csv("../Generador/Dataframe.csv", index_col = False)
df = df.drop(columns=["Unnamed: 0"])
df

,direccion,tipoVivienda,material_paredes,material_techo,material_piso,cuartos,banos,cocinas,comedores,dormitorios,personas_vivienda,jefe_hogar,departamento,provincia,distrito,date_time,ubigeo,fecha_censo,quien_respondio
0,Calle de Carolina Valle 502,Casa,Ladrillo,Eternit,Ceramica,6,2,1,1,1,2,Graciano Ayuso Mora,PIURA,MORROPON,BUENOS AIRES,2019-02-21 10:25:06,200402,07/07/2017,Graciano Ayuso Mora
1,Pasaje de Luis Miguel Olivera 594,Departamento,Ladrillo,Eternit,Parquet,2,3,1,0,1,2,Flavia Ribes Blanch,AYACUCHO,CANGALLO,PARAS,2010-05-25 02:47:37,50205,05/07/2017,Flavia Ribes Blanch
2,Camino de Delia Royo 485,Apartamento,Concreto,Techo de Paja,Ceramica,5,2,2,1,4,10,Ariel Miranda Cortés,MADRE DE DIOS,MANU,MADRE DE DIOS,2015-11-10 18:06:44,170203,10/07/2017,Ariel Miranda Cortés
3,Plaza Nuria Pera 812,Apartamento,Ladrillo,Techo de Paja,Ceramica,4,2,2,1,1,10,Amando Morante Navarro,SAN MARTIN,HUALLAGA,EL ESLABON,2015-04-23 05:16:18,220403,21/07/2017,Amando Morante Navarro
4,Cañada Olimpia Echevarría 904,Apartamento,Ladrillo,Techo de Paja,Parquet,1,4,1,0,1,2,Reyna Aznar-Alemán,ANCASH,HUAYLAS,PUEBLO LIBRE,1973-09-22 11:11:30,21207,01/07/2017,Reyna Aznar-Alemán
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,Plaza de Candelas Ugarte 686,Apartamento,Concreto,Eternit,Parquet,4,1,2,0,3,10,Noa Villena-Gibert,LIMA,LIMA,EL AGUSTINO,2000-08-11 13:46:31,150111,08/07/2017,Noa Villena-Gibert
49996,Via José Antonio Cañellas 371,Casa,Adobe,Teja,Parquet,6,4,1,2,2,2,Roldán Oliver-Ferrández,LAMBAYEQUE,CHICLAYO,SAÑA,2014-06-11 02:16:42,140115,02/07/2017,Roldán Oliver-Ferrández
49997,Ronda Raquel Patiño 806,Apartamento,Adobe,Techo de Paja,Parquet,1,1,2,2,1,2,José Ángel Conde-Moliner,CAJAMARCA,CHOTA,LAJAS,2015-03-27 04:18:03,60410,18/07/2017,José Ángel Conde-Moliner
49998,C. Juan Antonio Infante 245,Casa,Ladrillo,Teja,Parquet,5,4,1,1,4,1,Pepe Vilar Corbacho,LORETO,MAYNAS,ALTO NANAY,1985-09-06 07:41:51,160102,31/07/2017,Pepe Vilar Corbacho


In [2]:
G = nx.Graph()

G.add_node("Peru")

def accessDepartamento(dataFramePais, Grafo):
    Departamentos = list(dataFramePais["departamento"].unique())
    for departamento in Departamentos:
        Grafo.add_node(departamento)
        Grafo.add_edge("Peru", departamento)
        accessProvincia(dataFramePais, Grafo, departamento)

def accessProvincia(dataFrameDepartamento, Grafo, departamento):
    dataFrameProvincia = df.iloc[list(dataFrameDepartamento["departamento"] == departamento)]
    Provincias = list(dataFrameProvincia["provincia"].unique())
    for provincia in Provincias: 
        Grafo.add_node(provincia)
        Grafo.add_edge(departamento, provincia)
        accessDistrito(dataFrameProvincia, Grafo, provincia)

def accessDistrito(dataFrameProvincia, Grafo, provincia):
    dataFrameDistrito = dataFrameProvincia.iloc[list(dataFrameProvincia["provincia"] == provincia)]
    Distritos = list(dataFrameDistrito["distrito"].unique())
    for distrito in Distritos: 
        Grafo.add_node(distrito)
        Grafo.add_edge(provincia, distrito)
        accessDireccion(dataFrameDistrito, Grafo, distrito)

def accessDireccion(dataframeDistrito, Grafo, distrito):
    for index, row in dataframeDistrito.iterrows():
                direccion = row["direccion"]
                Grafo.add_node(direccion)
                Grafo.add_edge(distrito, direccion)

accessDepartamento(df, G)
nx.write_graphml_lxml(G,"test.graphml")